In [1]:
import matplotlib.pyplot as plt
import os
import random
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import skimage.transform as sktf
from PIL import Image

### Download data

In [2]:
%%bash

export KAGGLE_USERNAME='hung1578'
export KAGGLE_KEY='aa415e7d1ab043d0afe78e1dce04211d'
kaggle competitions download -c h-and-m-personalized-fashion-recommendations

100%|██████████| 28.7G/28.7G [02:12<00:00, 232MB/s]


In [3]:
!unzip /content/h-and-m-personalized-fashion-recommendations.zip

In [4]:
%rm /content/h-and-m-personalized-fashion-recommendations.zip

### Read data

In [40]:
articles = pd.read_csv("articles.csv", dtype={"article_id": "str"})
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_train.csv", dtype={"article_id": "str"})

In [41]:
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])

In [42]:
transactions = transactions.loc[transactions.t_dat >= pd.to_datetime('2020-08-22')]
transactions.shape

(1190911, 5)

### Sample data

In [52]:
sample = 0.05
customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False, random_state=45)
customers_sample_ids = set(customers_sample["customer_id"])
transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
articles_sample_ids = set(transactions_sample["article_id"])

In [53]:
# get article id which missed image
articles_id_ghost = []
print(len(customers_sample_ids),len(articles_sample_ids))
for i in articles_sample_ids:
    if not os.path.exists(f'images/{i[:3]}/{i}.jpg'):
        articles_id_ghost.append(i)

68599 11817


In [54]:
transactions_sample = transactions_sample.loc[~transactions_sample["article_id"].isin(articles_id_ghost)]

In [55]:
# check missing image
articles_sample_ids = set(transactions_sample["article_id"])
articles_id_ghost = []
print(len(customers_sample_ids),len(articles_sample_ids))
for i in articles_sample_ids:
    if not os.path.exists(f'images/{i[:3]}/{i}.jpg'):
        articles_id_ghost.append(i)
articles_id_ghost

68599 11800


[]

In [ ]:
os.mkdir('data')

In [57]:
if os.path.exists('data/images'):
    shutil.rmtree('data/images')
os.makedirs('data/images')

In [58]:
transactions_sample.shape

(59288, 5)

In [60]:
articles_sample_ids = set(transactions_sample["article_id"])
articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]

customers_sample_ids = set(transactions_sample["customer_id"])
customers_sample = customers[customers['customer_id'].isin(customers_sample_ids)]

customers_sample.to_csv("data/customers_sample.csv", index=False)
transactions_sample.to_csv("data/transactions_train_sample.csv", index=False)
articles_sample.to_csv("data/articles_train_sample.csv", index=False)

In [61]:
for i in tqdm(articles_sample_ids):
    if not os.path.exists(f'data/images/{i[:3]}/'): 
        os.mkdir(f'data/images/{i[:3]}/')

    im = plt.imread(f'images/{i[:3]}/{i}.jpg')
    im = sktf.resize(im,(540,360,3),mode='constant',preserve_range=True)
    im = (im-im.min())/(im.max()-im.min()) * 255.0
    im = im.astype(np.uint8)

    im = Image.fromarray(im)
    im.save(f'data/images/{i[:3]}/{i}.jpg')

100%|██████████| 11800/11800 [57:11<00:00,  3.44it/s]


In [62]:
shutil.make_archive('data', 'zip', '/content/data')

'/content/data.zip'

In [63]:
count = 0
for root_dir, cur_dir, files in os.walk('data/images'):
    count += len(files)
print('file count:', count)

file count: 11800


In [64]:
print(f'file size: {os.stat("data.zip").st_size / 1024 / 1024:0.2f} MB')

file size: 163.30 MB
